### Thread-safe Bags with Monitors (Java)

A bag (multiset) is a data structure in which the order of elements does not matter, like in a set, but in which an element can occur multiple times, like in a sequence. Here, we assume that all elements are integers:
- `insert(e)` adds integer `e` to the bag,
- `delete(e)` removes one occurrence of `e` from the bag, if there is one, otherwise does nothing,
- `has(e)` returns true if `e` occurs at least once in the bag.

The implementation below uses a singly linked list of nodes with a pointer to the first and last node. A bag always has an "empty" first node that does not contain an element but simplifies deletion.
- `insert(e)` adds a node with `e` at the end of the list,
- `delete(e)` removes the first node with `e`, assuming that a node with `e` exists, otherwise does nothing,
- `has(e)` returns true if a node with `e` is found.

In [1]:
%%writefile bags.java
class Node {
    int e;
    Node next = null;
}
class Bag {
    Node first, last; // first points to empty node
    // invariant: first != null && last != null && last.next == null &&
    // from first, following the next fields, last can be reached
    void print() {
        Node n = first;
        while (n != last) {
            n = n.next; System.out.print(n.e + " ");
        }
        System.out.println();
    }
    Bag() {
        first = new Node(); last = first;
    }
    void insert(int e) {
        Node n = new Node();
        n.e = e; last.next = n; last = n;
    }
    void delete(int e) {
        Node n = first;
        while (n != last && n.next.e != e) n = n.next;
        if (n != last) {
            if (n.next == last) last = n;
            n.next = n.next.next;
        }
    }
    boolean has(int e) {
        Node n = first;
        while (n != last && n.next.e != e) n = n.next;
        return n != last;
    }
}
class TestBag {
    public static void main(String[] args) {
        Bag b = new Bag();
        b.insert(5); b.print();
        b.insert(7); b.print();
        b.delete(7); b.print();
        System.out.println(b.has(5)); // false
        System.out.println(b.has(7)); // true
        System.out.println(b.has(3)); // false
    }
}

Writing bags.java


In [2]:
!javac bags.java

In [3]:
!java TestBag

5 
5 7 
5 
true
false
false


Suppose three kinds of processes access a bag concurrently: _inserters_ calling `insert`, _deleters_ calling `delete`, and _searchers_ calling `has`. Explain why class `Bag` is not thread-safe with an example!

Example:
Thread A which is calling insert(1)
Thread B which is called delete(1)

this may lead to race conditions since Thread B may not be aware of the update that Thread A made, this could lead to unexpected outcomes for what Thread B may delete. 

The following restrictions make the above implementation of bags thread-safe:
- An inserter must exclude other inserters and deleters
- A deleter must exclude other deleters, inserters, and searchers
- A searcher must exclude deleters

Modify the implementation below accordingly! State the class invariant of any fields that you introduce! The implementation below is *instrumented* with statements for *logging* the execution. These statements record in the field `log`, a string, whenever the actual insertion, deletion, and search start and end.

In [6]:
%%writefile threadsafebags.java
import java.util.Random;
import java.util.regex.Pattern;
class Node {
    int e;
    Node next = null;
}
class Bag {
    Node first, last; // first points to empty node
    String log = "";
    // invariant: first != null && last != null && last.next == null &&
    // from first, following the next fields, last can be reached
    Object insertLock = new Object();
    Object deleteLock = new Object();
    Object searchLock = new Object();
    void print() {
        Node n = first;
        while (n != last) {
            n = n.next; System.out.println(n.e);
        }
    }
    Bag() {
        first = new Node(); last = first;
    }
    void insert(int e) {
        synchronized (deleteLock) {
            synchronized (insertLock) {
                synchronized (log) {log += "I";}
                Node n = new Node();
                n.e = e; last.next = n; last = n;
                synchronized (log) {log += "i";}
            }
        }
    }
    void delete(int e) {
        synchronized (deleteLock) {
            synchronized (insertLock) {
                synchronized (searchLock) {
                    synchronized (log) {log += "D";}
                    Node n = first;
                    while (n != last && n.next.e != e) n = n.next;
                    if (n != last) {
                        if (n.next == last) last = n;
                        n.next = n.next.next;
                    }
                    synchronized (log) {log += "d";}
                }
            }
        }
    }
    boolean has(int e) {
        synchronized (deleteLock) {
            synchronized (log) {log += "H";}
            Node n = first;
            while (n != last && n.next.e != e) n = n.next;
            boolean found = n != last;
            synchronized (log) {log += "h";}
            return found;
        }
        
    }
}

class Inserter extends Thread {
    Bag b;
    int s;
    Inserter(Bag b, int s) {
        this.b = b; this.s = s;
    }
    public void run() {
        Random r = new Random();
        for (int i = 0; i < s; i++) {
            b.insert(r.nextInt(100));
        }
    }
}
class Deleter extends Thread {
    Bag b;
    int s;
    Deleter(Bag b, int s) {
        Random r = new Random();
        this.b = b; this.s = s;
    }
    public void run() {
        Random r = new Random();
        for (int i = 0; i < s; i++) {
            b.delete(r.nextInt(100));
        }
    }
}
class Searcher extends Thread {
    Bag b;
    int s;
    Searcher(Bag b, int s) {
        this.b = b; this.s = s;
    }
    public void run() {
        Random r = new Random();
        for (int i = 0; i < s; i++) {
            b.has(r.nextInt(100));
        }
    }
}
class TestBag {
    public static void main(String[] args) {
        final int P = 20; // number of inserter, deleter, and searcher threads
        final int S = 100; // number of repetitions by each thread
        Bag b = new Bag();
        Inserter[] in = new Inserter[P];
        Deleter[] dl = new Deleter[P];
        Searcher[] sr = new Searcher[P];
        for (int i = 0; i < P; i++) {
            in[i] = new Inserter(b, S); dl[i] = new Deleter(b, S); sr[i] = new Searcher(b, S);
        }
        for (int i = 0; i < P; i++) {
            in[i].start(); dl[i].start(); sr[i].start();
        }
        for (int i = 0; i < P; i++)
            try {in[i].join(); dl[i].join(); sr[i].join();} catch (Exception x) {}
        assert !Pattern.matches(".*I[IDd].*", b.log) : "An inserter must exclude other inserters and deleters";
        assert !Pattern.matches(".*D[DIiHh].*", b.log) : "A deleter must exclude other deleters, inserters, and searchers";
        assert !Pattern.matches(".*H[Dd].*", b.log) : "A searcher must exclude deleters";
        if (Pattern.matches(".*I[^i]*H.*", b.log)) System.out.println("Searcher concurrent with Inserter");
        if (Pattern.matches(".*H[^h]*I.*", b.log)) System.out.println("Inserter concurrent with Searcher");
        if (Pattern.matches(".*H[^h]*H.*", b.log)) System.out.println("Searcher concurrent with Searcher");
        System.out.println(b.log);
    }
}

Overwriting threadsafebags.java


The field `log` of class `Bag` is used for automated testing: `P` inserters, deleters, and searchers are started that insert, delete, or search for random values `S` times. [Regular expressions](https://docs.oracle.com/javase/tutorial/essential/regex/index.html) are used to check if the log contains only valid sequences of starts and ends of the bodies of `insert()`, `delete()`, and `has()`. Since a trivial implementation that allows these methods to run only in complete mutual exclusion would generate only valid logs, the three `if` statements at the end check for a truly concurrent execution. These checks only print the result and don't throw an exception, as it is theoretically possible that these checks fail because the scheduler did not attempt a truly concurrent execution.

In [7]:
!javac threadsafebags.java

In [8]:
!java -ea TestBag

IiIiHhHhHhDdDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiHhIiHhIiHhIiHhIiHhIiHhIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiDdDdDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiIiDdIiHhIiHhIiHhIiHhIiHhIiHhIiHhDdHhDdHhDdHhDdHhDdHhHhHhHhHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiIiHhIiHhIiHhIiHhIiHhHhHhDdDdDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiDdIiHhHhHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDdHhDd